## 반도체 소자 이상 탐지 AI 경진대회

- 알고리즘 ㅣ 월간데이콘 ㅣ 비전 ㅣ 비지도학습 ㅣ 이상 탐지 ㅣF1 Score

In [ ]:
# Dataset Info.

# train [폴더]
# 학습용 정상 반도체 소자 이미지 샘플 213개
# TRAIN_000.png ~ TRAIN_212.png


# test [폴더]
# 평가용 반도체 소자 이미지 샘플 100개
# TEST_000.png ~ TEST_099.png


# train.csv [파일] 
# ID : 샘플 ID
# img_path : 반도체 소자 이미지 파일 경로
# label : 0 (정상)


# test.csv [파일]
# ID : 샘플 ID
# img_path : 반도체 소자 이미지 파일 경로


# sample_submission.csv [파일] - 제출 양식
# ID : 샘플 ID
# label : 0 (정상), 1 (비정상)


# 데이터는 Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License (CC BY-NC-SA 4.0)에 따라 공개됩니다.

# Paul Bergmann, Kilian Batzner, Michael Fauser, David Sattlegger, Carsten Steger: The MVTec Anomaly Detection Dataset: A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: International Journal of Computer Vision 129(4):1038-1059, 2021, DOI: 10.1007/s11263-020-01400-4.

# Paul Bergmann, Michael Fauser, David Sattlegger, Carsten Steger: MVTec AD — A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection; in: IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR), 9584-9592, 2019, DOI: 10.1109/CVPR.2019.00982.



## Import Functions

In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm

# GPU 사용 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load Data

In [ ]:
# 데이터 로딩 클래스 정의

class CustomDataset(Dataset):
    
    def __init__(self, csv_file, transform=None):
        '''
        Args:
            csv_file (string) : csv 파일의 경로.
            transform (callable, optional) : 샘플에 적용될 Optional transform.
        '''
        
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df['img_path'].iloc[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image


# 이미지 전처리 및 임베딩
transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

train_data = CustomDataset(csv_file='./train.csv', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)





## Export Embedding Vector

In [ ]:
'''

이미지 데이터를 머신러닝 데이터에서 활용하기 위해 임베딩 벡터를 추출함.

이를 위해서 ImageNet 분류를 위해 만들어진 사전학습 모델인 ResNet-50을 활용함.

이런 ResNet-50 모델은 ImageNet에 존재하는 대량에 데이터로 학습되어 이미지에 존재하는 특정들에 대해서 잘 학습되어 있음. 

이 모델을 원래 의도된 분류를 위해 사용하지 않고, 모델이 가진 정보만을 이용해서 원하는 작업에 활용하는 것도 가능.

본 베이스라인의 경우 대회의 데이터셋에 대해서 ResNet-50 모델이 인식하는 정보를 추출하는 것이 목표이기 때문에, 

해당 모델의 마지막 레이어(분류 레이어)를 제거하고 ResNet-50이 대회 데이터셋에 대해서 인식하는 임베딩 벡터를 추출할 것.


'''

